# Reporte Técnico: Predicción de Abandono de Clientes (Churn)

Este documento detalla el proceso, las decisiones técnicas y los resultados obtenidos en el proyecto de predicción de churn para una empresa de telecomunicaciones.

---

## 1. Objetivo del Proyecto
El objetivo principal fue desarrollar un modelo capaz de identificar proactivamente a los clientes con alta probabilidad de cancelar su servicio, permitiendo así implementar estrategias de retención efectivas.

## 2. Metodología Paso a Paso

### 2.1 Exploración de Datos 
Se analizaron las distribuciones de las variables y su relación con la cancelación (Variable Objetivo - Churn):
- **Variables Numéricas**: Se observó que los clientes con mayor **MonthlyCharges** (cargos mensuales) y menor **tenure** (antigüedad) tienen una mayor tendencia a abandonar la empresa.
- **Variables Categóricas**: Se identificó que tipos de contrato específicos (Mes a Mes) y métodos de pago (Check electrónico) están fuertemente asociados con el churn.

### 2.2 Preprocesamiento de Datos
Para preparar los datos para los modelos, realizamos:
1. **Limpieza**: Eliminación de identificadores irrelevantes (`customerID`).
2. **Tratamiento de Nulos**: Se imputaron valores faltantes en `TotalCharges` usando la mediana.
3. **Codificación**:
   - **Binaria**: Variables con dos categorías se convirtieron en 0 y 1.
   - **Nuevas Categorías**: Categorías múltiples se transformaron en columnas booleanas independientes.
4. **Estandarización**: Se utilizó `StandardScaler` para que variables como `tenure` y `MonthlyCharges` estuvieran en la misma escala, evitando sesgos en el modelo.
5. **División**: Se dividió el set en 80% entrenamiento y 20% prueba, manteniendo la proporción original de churn (26%) en ambos para garantizar una evaluación justa.

---

## 3. Modelos Utilizados y Justificación

### Modelo 1: Regresión Logística (Modelo Base)
- **¿Qué es?**: Un modelo estadístico que predice la probabilidad de una variable binaria (0 o 1).
- **Justificación**: Es simple, rápido de entrenar y altamente interpretable (permite ver qué variables pesan más). Sirve como punto de comparación.
- **Configuración**: Se usó `class_weight='balanced'` para compensar que hay menos casos de churn que de retención.

### Modelo 2: XGBoost (Optimizado)
- **¿Qué es?**: Un algoritmo de *Gradient Boosting* basado en árboles de decisión que construye modelos en serie, donde cada nuevo árbol corrige los errores de los anteriores.
- **Justificación**: Es uno de los algoritmos más potentes actualmente debido a su manejo de relaciones no lineales y su resistencia al sobreajuste (*overfitting*).
- **Optimización**: Se realizó una búsqueda aleatoria de hiperparámetros (`RandomizedSearchCV`) con validación cruzada para encontrar la mejor combinación de profundidad de árboles y velocidad de aprendizaje.

---

## 4. Resultados y Comparativa

Se evaluaron los modelos principalmente mediante el **ROC-AUC** (capacidad de distinguir entre clases) y el **Recall** (capacidad de detectar a los que sí se van).

| Métrica | Regresión Logística | XGBoost |
| :--- | :---: | :---: |
| **ROC-AUC** | 0.8417 | **0.8468** |
| **Recall (Churn)** | 0.78 | **0.81** |
| **Precisión (Churn)** | 0.51 | **0.52** |
| **Accuracy** | 0.74 | **0.75** |

### Conclusiones de Resultados:
- **XGBoost** superó ligeramente a la Regresión Logística en todas las métricas, confirmando ser el modelo más robusto.
- **Estrategia de Trade-off (Precisión vs. Recall)**: 
    - Hemos priorizado intencionalmente el **Recall (0.81)** sobre la precisión.
    - **Razón de Negocio**: En la predicción de churn, el costo de omitir a un cliente que realmente se va (Falso Negativo) es significativamente mayor que el costo de contactar a un cliente que no tenía intención de irse (Falso Positivo).
    - Con una precisión del **52%**, sabemos que aproximadamente la mitad de nuestras alertas serán "falsas alarmas", pero esto nos permite capturar el **81% de los abandonos reales**, maximizando la retención de ingresos.
- Ambos modelos muestran un excelente **ROC-AUC (0.84+)**, lo que indica una alta capacidad para distinguir entre clientes que cancelan y los que permanecen.